In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, RandomizedSearchCV, GridSearchCV, GroupKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, confusion_matrix
import time
from sklearn.metrics import precision_score, make_scorer,recall_score,f1_score,roc_auc_score
import warnings
import csv
from os import listdir, chdir
from os.path import isfile, join

In [2]:
def hyperparameter_tune(base_model, parameters, kfold, X, y, groups):
    start_time = time.time()
    k = GroupKFold(n_splits=kfold)

    prec_1 = make_scorer(precision_score, pos_label=1)
    rec_1 = make_scorer(recall_score, pos_label=1)
    f1_1 = make_scorer(f1_score, pos_label=1)
    roc = make_scorer(roc_auc_score)
    prec_0 = make_scorer(precision_score, pos_label=0)
    rec_0 = make_scorer(recall_score, pos_label=0)
    f1_0 = make_scorer(f1_score, pos_label=0)

    scoring_st = {'prec_1': prec_1, 'rec_1': rec_1, 'f1_1': f1_1, 'roc': roc, 'prec_0': prec_0, 'rec_0': rec_0,
              'f1_0': f1_0}

    optimal_model = RandomizedSearchCV(base_model,
                                      param_distributions=parameters,
                                      n_iter=200,
                                      cv=k,
                                      scoring = scoring_st,
                                      n_jobs=10,
                                      refit='rec_1',
                                      verbose=3,
                                      return_train_score=True)
                                      #random_state=SEED)

    optimal_model.fit(X, y, groups)

    stop_time = time.time()
    #scores = cross_validate(optimal_model, X, y, cv=k, scoring= scoring_st, return_train_score=True, return_estimator=True)
    print("Elapsed Time:", time.strftime("%H:%M:%S", time.gmtime(stop_time - start_time)))
    print("====================")
    #print("Cross Val Mean: {:.3f}, Cross Val Stdev: {:.3f}".format(scores.mean(), scores.std()))
    print("Best Score: {:.3f}".format(optimal_model.best_score_))
    print("Best Parameters: {}".format(optimal_model.best_params_))
    return optimal_model.best_params_, optimal_model.best_score_, optimal_model.cv_results

In [3]:
mypath = '/work2/pa21/sgirtsou/production/datasets/hard_cosine_similarity'

In [4]:
allfiles = [f for f in listdir(mypath) if f.endswith('norm.csv') and f[0].isdigit()]

In [5]:
allfiles

['2016_norm.csv',
 '2015_norm.csv',
 '2014_norm.csv',
 '2017_norm.csv',
 '2012_norm.csv',
 '2011_norm.csv',
 '2010_norm.csv',
 '2018_norm.csv',
 '2013_norm.csv']

In [6]:
chdir(mypath)

In [7]:
li = []
for filename in allfiles:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    
frame = pd.concat(li, axis=0, ignore_index=True)

In [8]:
frame.firedate = pd.to_datetime(frame.firedate).dt.strftime('%Y%m%d')

In [9]:
df_part = frame[['id', 'firedate', 'max_temp', 'min_temp', 'mean_temp','res_max', 'dom_vel', 'rain_7days', 'dem', 'slope', 'curvature',
       'aspect', 'ndvi_new', 'evi', 'lst_day', 'lst_night', 'max_dew_temp','mean_dew_temp', 'min_dew_temp', 'fire', 'dir_max_1', 'dir_max_2',
       'dir_max_3', 'dir_max_4', 'dir_max_5', 'dir_max_6', 'dir_max_7','dir_max_8', 'dom_dir_1', 'dom_dir_2', 'dom_dir_3', 'dom_dir_4',
       'dom_dir_5', 'dom_dir_6', 'dom_dir_7', 'dom_dir_8', 'corine_111','corine_112', 'corine_121', 'corine_122', 'corine_123', 'corine_124',
       'corine_131', 'corine_132', 'corine_133', 'corine_141', 'corine_142','corine_211', 'corine_212', 'corine_213', 'corine_221', 'corine_222',
       'corine_223', 'corine_231', 'corine_241', 'corine_242', 'corine_243','corine_244', 'corine_311', 'corine_312', 'corine_313', 'corine_321',
       'corine_322', 'corine_323', 'corine_324', 'corine_331', 'corine_332','corine_333', 'corine_334', 'corine_411', 'corine_412', 'corine_421',
       'corine_422', 'corine_511', 'corine_512', 'corine_521', 'wkd_0','wkd_1', 'wkd_2', 'wkd_3', 'wkd_4', 'wkd_5', 'wkd_6', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'month_4', 'frequency','f81', 'x', 'y']].copy()

X_unnorm, y_int = df_part[['max_temp', 'min_temp', 'mean_temp','res_max', 'dom_vel', 'rain_7days', 'dem', 'slope', 'curvature',
       'aspect', 'ndvi_new', 'evi', 'lst_day', 'lst_night', 'max_dew_temp','mean_dew_temp', 'min_dew_temp', 'dir_max_1', 'dir_max_2',
       'dir_max_3', 'dir_max_4', 'dir_max_5', 'dir_max_6', 'dir_max_7','dir_max_8', 'dom_dir_1', 'dom_dir_2', 'dom_dir_3', 'dom_dir_4',
       'dom_dir_5', 'dom_dir_6', 'dom_dir_7', 'dom_dir_8', 'corine_111','corine_112', 'corine_121', 'corine_122', 'corine_123', 'corine_124',
       'corine_131', 'corine_132', 'corine_133', 'corine_141', 'corine_142','corine_211', 'corine_212', 'corine_213', 'corine_221', 'corine_222',
       'corine_223', 'corine_231', 'corine_241', 'corine_242', 'corine_243','corine_244', 'corine_311', 'corine_312', 'corine_313', 'corine_321',
       'corine_322', 'corine_323', 'corine_324', 'corine_331', 'corine_332','corine_333', 'corine_334', 'corine_411', 'corine_412', 'corine_421',
       'corine_422', 'corine_511', 'corine_512', 'corine_521', 'wkd_0','wkd_1', 'wkd_2', 'wkd_3', 'wkd_4', 'wkd_5', 'wkd_6', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'month_4', 'frequency','f81', 'x', 'y']], df_part['fire']

groups = frame['firedate']

#X = normalize_dataset(X_unnorm, 'std')
y = y_int

X_ = X_unnorm.values
#X_ = X.values
y_ = y.values
groupskfold = groups.values

rf = RandomForestClassifier(n_jobs=-1)
depth = [10, 20, 100, 200, 400,500, 700, 1000, 1200,2000, None]
n_estimators = [50, 100, 120, 150,170,200, 250, 350, 500, 750, 1000,1400, 1500]
min_samples_split = [2, 10, 50, 70,100,120,150,180, 200, 250,400,600,1000, 1300, 2000]
min_samples_leaf = [1, 10,30,40,50,100,120,150] #with numbers
max_features = list(range(1,X_.shape[1]))
bootstrap = [True, False]
criterion = ["gini", "entropy"]
class_weights = [{0:1,1:9},{0:1,1:300},{0:1,1:400},{0:1,1:500},{0:1,1:1000}]

In [10]:
lots_of_parameters = {
    "max_depth": depth, #depth of each tree
    "n_estimators": n_estimators, #trees of the forest
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf,
    "criterion": criterion,
    "max_features": max_features,
    "bootstrap": bootstrap,
    "class_weight": class_weights
}


best_scores = []
best_parameters = []
full_scores = []
folds = [10]#range(2, 8)

columns_sel = ['mean_test_acc','std_test_acc', 'mean_train_acc', 'std_train_acc','mean_test_AUC','std_test_AUC', 'mean_train_AUC', 'std_train_AUC',
              'mean_test_prec','std_test_prec','mean_train_prec','std_train_prec', 'mean_test_rec','std_test_rec',
              'mean_train_rec','std_train_rec', 'mean_test_f_score','std_test_f_score', 'mean_train_f_score','std_train_f_score',
               'params','folds']

results = pd.DataFrame(columns=columns_sel)

for i in folds:
    print("\ncv = ", i)
    start = time.time()
    best_params, best_score, full_scores = hyperparameter_tune(rf, lots_of_parameters, i, X_, y_, groupskfold)

    df_results = pd.DataFrame.from_dict(full_scores)
    df_results['folds'] = int(i)
    #df_results.to_csv('/home/sgirtsou/Documents/GridSearchCV/RF/RFcv_25kbalanced_noshufflestrictcriterion.csv')

    df_short.to_csv('rf_random_search.csv')


cv =  10
Fitting 10 folds for each of 200 candidates, totalling 2000 fits
[CV] n_estimators=200, min_samples_split=180, min_samples_leaf=50, max_features=25, max_depth=100, criterion=gini, class_weight={0: 1, 1: 9}, bootstrap=False 
[CV] n_estimators=200, min_samples_split=180, min_samples_leaf=50, max_features=25, max_depth=100, criterion=gini, class_weight={0: 1, 1: 9}, bootstrap=False 
[CV] n_estimators=200, min_samples_split=180, min_samples_leaf=50, max_features=25, max_depth=100, criterion=gini, class_weight={0: 1, 1: 9}, bootstrap=False 
[CV] n_estimators=200, min_samples_split=180, min_samples_leaf=50, max_features=25, max_depth=100, criterion=gini, class_weight={0: 1, 1: 9}, bootstrap=False 
[CV] n_estimators=200, min_samples_split=180, min_samples_leaf=50, max_features=25, max_depth=100, criterion=gini, class_weight={0: 1, 1: 9}, bootstrap=False 
[CV] n_estimators=200, min_samples_split=180, min_samples_leaf=50, max_features=25, max_depth=100, criterion=gini, class_weight={0

JoblibRuntimeError: JoblibRuntimeError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/apps/applications/python/anaconda3/5.0.1/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/apps/

In [14]:
frame.to_csv('full_dataset_norm.csv')

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/work2/pa21/sgirtsou/production/datasets/hard_cosine_similarity/full_dataset_norm.csv')

In [4]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'firedate', 'max_temp', 'min_temp',
       'mean_temp', 'res_max', 'dom_vel', 'rain_7days', 'dem', 'slope',
       'curvature', 'aspect', 'ndvi_new', 'evi', 'lst_day', 'lst_night',
       'max_dew_temp', 'mean_dew_temp', 'min_dew_temp', 'fire', 'dir_max_1',
       'dir_max_2', 'dir_max_3', 'dir_max_4', 'dir_max_5', 'dir_max_6',
       'dir_max_7', 'dir_max_8', 'dom_dir_1', 'dom_dir_2', 'dom_dir_3',
       'dom_dir_4', 'dom_dir_5', 'dom_dir_6', 'dom_dir_7', 'dom_dir_8',
       'corine_111', 'corine_112', 'corine_121', 'corine_122', 'corine_123',
       'corine_124', 'corine_131', 'corine_132', 'corine_133', 'corine_141',
       'corine_142', 'corine_211', 'corine_212', 'corine_213', 'corine_221',
       'corine_222', 'corine_223', 'corine_231', 'corine_241', 'corine_242',
       'corine_243', 'corine_244', 'corine_311', 'corine_312', 'corine_313',
       'corine_321', 'corine_322', 'corine_323', 'corine_324', 'corine_331',
       'corine_332', 